In [ ]:
import requests
from bs4 import BeautifulSoup as bs

In [ ]:
# 실제 사이트에서 html정보를 받아와 변수에 저장

data = requests.get("https://finance.naver.com/marketindex/exchangeList.naver").text

In [ ]:
# BeautifulSoup 이용해 데이터 파싱
soup = bs(data,"html.parser")

In [ ]:
# bs.find() -> html안에서 해당 태그의 첫번째 정보를 출력
tbody = soup.find("tbody")
tbody

In [ ]:
# bs.find_all() -> 해당 태그 정보 전체를 리스트 형태로 출력
# 항목의 "\t","\n" 다 지워주기
# soup.find_all("td")[0].get_text().replace("\t","").strip("\n")

td_list = soup.find_all("td")

In [ ]:
# td_list를 이용해 2차원 배열로 새로운 리스트 생성
# step1) "\t","\n" 제거해서 1차원 배열에 담기
td_list_1= []

for i in td_list:
    i = i.get_text().replace("\t","")
    i = i.strip("\n")
    td_list_1.append(i)

td_list_1

In [ ]:
# step2) ['미국 USD','1,315.50', '1,338.52', '1,292.48', '1,328.30', '1,302.70', '1.000']를 하나의 행으로
# 각 나라 데이터 2차원 배열로 표현
# 방법1) 2중for문
td_list_2 = []

for i in range (0,len(td_list_1)//7):
    _row = []
    for j in range(i*7,i*7+7):
        _row.append(td_list_1[j])
    
    td_list_2.append(_row)


td_list_2

In [ ]:
# 방법2) 슬라이싱
td_list_2 = []

for i in range (0,len(td_list_1)//7):
    j = i * 7
    _row =td_list_1[j:j+7]
    
    td_list_2.append(_row)


td_list_2

In [ ]:
# 생성한 배열 데이터프레임으로 만들기
import pandas as pd

In [ ]:
columns = ["통화명","매매기준율","현찰(구매)","현찰(판매)","송금(보낼때)","송금(받을때)","미화환산율"]
data = pd.DataFrame(td_list_2,columns=columns)

In [ ]:
data.to_csv("../csv/naver.csv",index=False)

In [ ]:
import mod_sql

In [ ]:
_db = mod_sql.Database("localhost","root","1234","ubion7",3306)

### 실습
1. DB에 naver table생성
2. 필드는 7개 모두 문자열 타입
3. 통화명 필드는 primary key

In [ ]:
sql = """
    create table naver(
    `통화명` varchar(16) primary key,
    `매매기준율` varchar(16),
    `현찰(구매)` varchar(16),
    `현찰(판매)` varchar(16),
    `송금(보낼때)` varchar(16),
    `송금(받을때)` varchar(16),
    `미화환산율` varchar(16)
    )
    """

_db.sql_query(sql)

#### naver라는 이름으로 생성한 테이블에 데이터 삽입
#### 데이터를 삽입하는 insert문 작성

In [49]:
sql = """
    insert into naver
    values (%s,%s,%s,%s,%s,%s,%s)
"""

for i in td_list_2:
    _db.sql_query(sql,i)

In [48]:
sql = """
    truncate naver
"""

_db.sql_query(sql)

'Query OK'

In [50]:
sql = """
    select
    *
    from naver
"""

_db.sql_query(sql)

,통화명,매매기준율,현찰(구매),현찰(판매),송금(보낼때),송금(받을때),미화환산율
0,남아프리카 공화국 ZAR,72.58,78.38,66.78,73.45,71.71,0.055
1,네팔 NPR,10.04,N/A,N/A,N/A,N/A,0.008
2,노르웨이 NOK,125.74,128.82,122.66,126.99,124.49,0.095
3,뉴질랜드 NZD,818.45,834.57,802.33,826.63,810.27,0.621
4,대만 TWD,43.20,48.85,38.88,N/A,N/A,0.033
5,덴마크 DKK,194.18,198.93,189.43,196.12,192.24,0.147
6,러시아 RUB,16.16,17.69,13.34,16.72,15.60,0.012
7,루마니아 RON,293.13,N/A,N/A,N/A,N/A,0.222
8,리비아 LYD,276.64,N/A,N/A,279.40,273.88,0.210
9,마카오 MOP,163.00,N/A,N/A,N/A,N/A,0.124


In [56]:
sql = """
    delete
    from naver2;
"""

_db.sql_query(sql)

'Query OK'

In [58]:
sql = """
    insert into naver2
    (`통화명`,`매매기준율`,`현찰(구매)`,`현찰(판매)`,`송금(보낼때)`,`송금(받을때)`,`미화환산율`)				
    values (%s,%s,%s,%s,%s,%s,%s)
"""

for i in td_list_2:
    _db.sql_query(sql,i)

In [59]:
sql = """
    select
    *
    from naver2
"""

_db.sql_query(sql)

,id,통화명,매매기준율,현찰(판매),현찰(구매),송금(보낼때),송금(받을때),미화환산율
0,59,미국 USD,"1,318.80","1,295.73","1,341.87","1,331.70","1,305.90",1.000
1,60,유럽연합 EUR,"1,447.12","1,418.33","1,475.91","1,461.59","1,432.65",1.097
2,61,일본 JPY (100엔),981.80,964.62,998.98,991.42,972.18,0.745
3,62,중국 CNY,191.61,182.03,201.19,193.52,189.70,0.145
4,63,홍콩 HKD,168.01,164.71,171.31,169.69,166.33,0.127
5,64,대만 TWD,43.20,38.88,48.85,N/A,N/A,0.033
6,65,영국 GBP,"1,637.82","1,605.56","1,670.08","1,654.19","1,621.45",1.242
7,66,오만 OMR,"3,425.28","3,219.77","3,730.12",N/A,N/A,2.597
8,67,캐나다 CAD,984.51,965.12,"1,003.90",994.35,974.67,0.747
9,68,스위스 CHF,"1,470.48","1,441.52","1,499.44","1,485.18","1,455.78",1.115
